In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import xarray as xr
import dask
import zarr

In [3]:
main = xr.open_zarr('/mnt/efs/data/ecco/zarr/eccov4r3_output/')

In [4]:
budgFw = xr.open_zarr('/mnt/efs/data/ecco/zarr/anom/budgFw/')

### Depth average (upper Z)

In [5]:
budgFw.coords['Z'] = main.Z

In [6]:
np.set_printoptions(formatter={"float_kind": lambda x: "%g" % x})
print(budgFw.Z.values)

[-5 -15 -25 -35 -45 -55 -65 -75.005 -85.025 -95.095 -105.31 -115.87
 -127.15 -139.74 -154.47 -172.4 -194.735 -222.71 -257.47 -299.93 -350.68
 -409.93 -477.47 -552.71 -634.735 -722.4 -814.47 -909.74 -1007.16 -1105.91
 -1205.54 -1306.2 -1409.15 -1517.09 -1634.18 -1765.14 -1914.15 -2084.03
 -2276.23 -2491.25 -2729.25 -2990.25 -3274.25 -3581.25 -3911.25 -4264.25
 -4640.25 -5039.25 -5461.25 -5906.25]


In [7]:
budgFw = budgFw.swap_dims({'k':'Z'})

In [8]:
z_lower = 20
print('Depth at lower cell edge:',main.Zu[budgFw.sel(Z=slice(0,-z_lower)).k.values[-1]].values,' m')

Depth at lower cell edge: -20.0  m


In [9]:
# Create mask to ensure all terms have the depth range in defined values
mask = budgFw.frc + budgFw.adv_ConvFWb + budgFw.adv_ConvFWe + budgFw.adv_ConvFw + \
        budgFw.hadv_Uanom_FWanom + budgFw.hadv_Uanom_FWclim + budgFw.hadv_Uclim_FWanom + budgFw.tnd + \
        budgFw.vadv_Uanom_FWanom + budgFw.vadv_Uanom_FWclim + budgFw.vadv_Uclim_FWanom

In [10]:
#budgFw_aveZ = (budgFw*(mask/mask)).sel(Z=slice(0,-z_lower)).sum('Z')/budgFw.sel(Z=slice(0,-z_lower)).vol.sum('Z')
budgFw_sumZ = (budgFw*(mask/mask)).sel(Z=slice(0,-z_lower)).sum('Z')

### Rechunking

In [11]:
for varname in budgFw_sumZ.data_vars:
    budgFw_sumZ[varname] = budgFw_sumZ[varname].chunk(chunks={'time':288,'face':1,'j':90,'i':90})

In [12]:
budgFw_sumZ = budgFw_sumZ.reset_coords(drop=True)

In [13]:
budgFw_sumZ

<xarray.Dataset>
Dimensions:            (face: 13, i: 90, j: 90, time: 288)
Coordinates:
  * face               (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i                  (i) int64 0 1 2 3 4 5 6 7 8 ... 82 83 84 85 86 87 88 89
  * j                  (j) int64 0 1 2 3 4 5 6 7 8 ... 82 83 84 85 86 87 88 89
  * time               (time) datetime64[ns] 1992-01-15 ... 2015-12-14T12:00:00
Data variables:
    adv_ConvFWb        (time, face, j, i) float64 dask.array<chunksize=(288, 1, 90, 90), meta=np.ndarray>
    adv_ConvFWe        (time, face, j, i) float64 dask.array<chunksize=(288, 1, 90, 90), meta=np.ndarray>
    adv_ConvFw         (time, face, j, i) float64 dask.array<chunksize=(288, 1, 90, 90), meta=np.ndarray>
    frc                (time, face, j, i) float64 dask.array<chunksize=(288, 1, 90, 90), meta=np.ndarray>
    hadv_Uanom_FWanom  (time, face, j, i) float64 dask.array<chunksize=(288, 1, 90, 90), meta=np.ndarray>
    hadv_Uanom_FWclim  (time, face, j, i) float64 dask.array<c

### Save to zarr

In [14]:
compressor = zarr.Blosc(cname='zlib')
opts = {}
for varname in budgFw_sumZ.data_vars:
    opts[varname] = {'compressor': compressor, 'chunks': (288, 1, 90, 90)}

In [15]:
from dask.diagnostics import ProgressBar

In [16]:
ls /mnt/efs/data/ecco/zarr/anom

budgFw/  budgH/


In [17]:
with ProgressBar():
    budgFw_sumZ.to_zarr('/mnt/efs/data/ecco/zarr/anom/budgFw_20m', encoding=opts)

[########################################] | 100% Completed | 41.6s
